# 0. Installing the pre-requisted libs

In [1]:
!pip install beautifulsoup4 requests pandas geocoder

     |████████████████████████████████| 102kB 15.0MB/s ta 0:00:01


# 1. Data scrape

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import geocoder
import time
import json

### 1.1 Scraping from wikipedia

In [3]:
def link_or_text(elem):
    link = elem.select_one('a')
    if link:
        return link.text.strip()
    return elem.text.strip()

In [61]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
pc_data = []

html = requests.get(URL).content
soup = BeautifulSoup(html, 'html.parser')
tbody = soup.select_one('table.wikitable>tbody')

for tr in tbody.find_all('tr'):
    tds = tr.find_all('td')
    if len(tds) == 3:
        pc, borough, neighbourhood = [link_or_text(td) for td in tds]
        if borough == 'Not assigned':
            # ignore when borough is ''Not assigned
            continue
        item = dict(zip(['Postcode', 'Borough', 'Neighbourhood'],  [pc, borough, neighbourhood]))
        pc_data.append(item) 

In [62]:
pc_data[:3]

[{'Postcode': 'M3A', 'Borough': 'North York', 'Neighbourhood': 'Parkwoods'},
 {'Postcode': 'M4A',
  'Borough': 'North York',
  'Neighbourhood': 'Victoria Village'},
 {'Postcode': 'M5A',
  'Borough': 'Downtown Toronto',
  'Neighbourhood': 'Harbourfront'}]

In [64]:
pc_df = pd.DataFrame(pc_data)

In [65]:
pc_df.head()

,Borough,Neighbourhood,Postcode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,Harbourfront,M5A
3,Downtown Toronto,Regent Park,M5A
4,North York,Lawrence Heights,M6A


In [66]:
pc_df.tail()

,Borough,Neighbourhood,Postcode
206,Etobicoke,Kingsway Park South West,M8Z
207,Etobicoke,Mimico NW,M8Z
208,Etobicoke,The Queensway West,M8Z
209,Etobicoke,Royal York South West,M8Z
210,Etobicoke,South of Bloor,M8Z


### 1.2 Retriving location coordinates

In [67]:
GCP_API_KEY = 'THIS_IS_A_SECRET'

In [68]:
# The code was removed by Watson Studio for sharing.

In [69]:
latlong_data = []

qry_result = []
for item in pc_data:
    pc = item.get('Postcode')
    qry = f'{pc}, Toronto, Ontario'
    for i in range(3):
        r = geocoder.google(qry, key=GCP_API_KEY)
        if r.latlng:
            #print(f'{qry} {r.latlng}')
            qry_result.append(r)
            break
        else:
            time.sleep(1.5)
    

In [70]:
## `latlongs` is just a list of dictionary with keys Latitude and Longtitude 
## and we will merge it back to the pc_data later
latlongs = [dict([('Latitude', r.latlng[0]), ('Longtitude', r.latlng[1])])  for r in  qry_result]

In [73]:
len(latlongs), len(pc_data)

(211, 211)

In [74]:
latlongs[:5]

[{'Latitude': 43.7532586, 'Longtitude': -79.3296565},
 {'Latitude': 43.72588229999999, 'Longtitude': -79.3155716},
 {'Latitude': 43.6542599, 'Longtitude': -79.36063589999999},
 {'Latitude': 43.6542599, 'Longtitude': -79.36063589999999},
 {'Latitude': 43.718518, 'Longtitude': -79.4647633}]

In [75]:
## Merging the pc_data and latlongs
_ = [d.update(d_latlng) for (d, d_latlng) in zip(pc_data, latlongs)]

In [76]:
pc_data[:3]

[{'Postcode': 'M3A',
  'Borough': 'North York',
  'Neighbourhood': 'Parkwoods',
  'Latitude': 43.7532586,
  'Longtitude': -79.3296565},
 {'Postcode': 'M4A',
  'Borough': 'North York',
  'Neighbourhood': 'Victoria Village',
  'Latitude': 43.72588229999999,
  'Longtitude': -79.3155716},
 {'Postcode': 'M5A',
  'Borough': 'Downtown Toronto',
  'Neighbourhood': 'Harbourfront',
  'Latitude': 43.6542599,
  'Longtitude': -79.36063589999999}]

In [77]:
df_pc2 = pd.DataFrame(pc_data, columns=['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longtitude'])

In [78]:
df_pc2

,Postcode,Borough,Neighbourhood,Latitude,Longtitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
6,M7A,Queen's Park,Not assigned,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,Rouge,43.806686,-79.194353
9,M1B,Scarborough,Malvern,43.806686,-79.194353
